# Topic Modeling

**Importing needed libraries**

In [20]:
import re

import pyLDAvis.gensim_models
from gensim.models.ldamulticore import LdaMulticore
import warnings

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

**Topic Modeling with Alic in Wondeland Text**

- Prepare text: Load text file, split into documents, tokenize/lemmatize, remove stop words
- Create the term dictionary for the corpus
- Create a document term matrix (DTM)
- Set up the LDA model, decide on the number of topics
- Run and train the model
- Topics!

In [3]:
# Function for tokenizing the 
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [7]:
# Read in the text (download to run locally)
with open('data/alice_in_wonderland.txt', 'r', encoding="utf8") as file:
    text_str = file.read()

# Split the string on the newline character
text = text_str.split('\n')

# Tokenize each chunk of text
text_tokens = [tokenize(chunk) for chunk in text]

# Look at the first 10 tokens
text_tokens[0][0:10]

['alice', 'adventures', 'wonderland']

In [9]:
# Create the term dictionary of our corpus
# every unique term is assigned an index
dictionary = corpora.Dictionary(text_tokens)

# Convert list of documents (corpus) into Document Term Matrix 
# using the dictionary we just created
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# What does this matrix look like?
print(doc_term_matrix[0][0:25])

[(0, 1), (1, 1), (2, 1)]


In [11]:
# Create the object for LDA model
lda = gensim.models.ldamodel.LdaModel

# Train LDA model on the document term matrix
# topics = 5
ldamodel = lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

# Print out the topics
print(ldamodel.print_topics(num_topics=3, num_words=5))

[(2, '0.031*"queen" + 0.015*"king" + 0.014*"know" + 0.014*"gryphon" + 0.011*"like"'), (0, '0.030*"alice" + 0.019*"think" + 0.017*"like" + 0.016*"works" + 0.011*"rabbit"'), (4, '0.112*"said" + 0.060*"alice" + 0.014*"turtle" + 0.013*"hatter" + 0.013*"mock"')]


In [13]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in ldamodel.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
alice think like works rabbit

------ Topic 1 ------
little alice voice went terms

------ Topic 2 ------
queen king know gryphon like

------ Topic 3 ------
gutenberg project tm work electronic

------ Topic 4 ------
said alice turtle hatter mock



**Interpreting Topics**

In [19]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Use the visualization in a notebook
pyLDAvis.enable_notebook()

In [24]:
# Function for tokenizing the 
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

# Read in the text (download to run locally)
with open('data/alice_in_wonderland.txt', 'r', encoding="utf8") as file:
    text_str = file.read()

# Split the string on the newline character
text = text_str.split('\n')

# Tokenize each chunk of text
text_tokens = [tokenize(chunk) for chunk in text]

# Create the term dictionary of our corpus
# every unique term is assigned an index
dictionary = corpora.Dictionary(text_tokens)

# Convert list of documents (corpus) into Document Term Matrix 
# using the dictionary we just created
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# Create the object for LDA model
lda = gensim.models.ldamodel.LdaModel

# Train LDA model on the document term matrix
# topics = 5
ldamodel = lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

In [27]:
# Interactive visualization for topic modeling
# (a screenshot will be displayed below)
pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)

C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.138078 -0.043893       1        1  26.885360
3      0.093600 -0.067782       2        1  21.246432
0     -0.282354 -0.044872       3        1  18.062132
2      0.011403  0.230095       4        1  17.925188
4      0.039273 -0.073549       5        1  15.880888, topic_info=           Term        Freq       Total Category  logprob  loglift
134        said  381.000000  381.000000  Default  30.0000  30.0000
1         alice  320.000000  320.000000  Default  29.0000  29.0000
2318  gutenberg   84.000000   84.000000  Default  28.0000  28.0000
2319    project   76.000000   76.000000  Default  27.0000  27.0000
2355         tm   51.000000   51.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
162       began    8.671340   47.345242   Topic5  -5.3861   0.1426
189      little    8.843724   98.126316   Topic5  -5.3664  -0.5665
237        long    7.681532   26.493881   Topic5  -5.5073   0.6020
300        poor    7.447854   21.803217   Topic5  -5.5382   0.7659
177        know    7.808676   70.837861   Topic5  -5.4909  -0.3651

[277 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2393      3  0.922457      access
874       1  0.097513       added
874       2  0.633832       added
874       5  0.243782       added
0         5  0.966016  adventures
...     ...       ...         ...
1572      5  0.889939     writing
190       3  0.954021     written
547       5  0.821445       wrong
2329      3  0.902710         www
154       1  0.929574         yes

[334 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])